# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  0,  1,  1],
       [ 0,  2,  1,  1],
       [ 0, 15,  1,  1],
       [ 0, 39,  1,  1],
       [ 0, 51,  1,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 2095,    0,    4],
       [5795, 2180,    0,    2],
       [5795, 2215,    0,    1],
       [5795, 2234,    0,    1],
       [5795, 2293,    0,    1]])

In [7]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 267241
Nr of rows in test file 109004


In [8]:
print('Nr of emails in training file', np.unique(sparse_train_data[:, 0]).size)

Nr of emails in training file 4015


In [9]:
print('Nr of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Nr of emails in test file 1722


### How to create an Empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795], shape=(4015,))

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

/tmp/ipykernel_6206/3253082390.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_train_data.fillna(value=0, inplace=True)


In [14]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
sparse_train_data[10:13]

array([[   0,  161,    1,    1],
       [   0,  180,    1,    1],
       [   0, 1350,    1,    1]])

In [16]:
sparse_train_data[10][3]

np.int64(1)

# Create a Full Matrix from a Sparse Matrix

In [55]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    with pd.option_context('future.no_silent_downcasting', True):
    # Do you thing with fillna, ffill, bfill, replace...
    # and possible use infer_objects if needed
    
        column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
        doc_id_names = np.unique(sparse_matrix[:, 0])
        full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
        full_matrix.fillna(value=0, inplace=True)
        
        for i in range(sparse_matrix.shape[0]):
            doc_nr = sparse_matrix[i][doc_idx]
            word_id = sparse_matrix[i][word_idx]
            label = sparse_matrix[i][cat_idx]
            occurrence = sparse_matrix[i][freq_idx]
            
            full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
            full_matrix.at[doc_nr, 'CATEGORY'] = label
            full_matrix.at[doc_nr, word_id] = occurrence
        
        full_matrix.set_index('DOC_ID', inplace=True)
        return full_matrix
    

In [56]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 26.2 s, sys: 194 ms, total: 26.4 s
Wall time: 26.6 s


In [57]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,1,3,2,0,3,0,2,3,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,2,0,2,0,0,1,1,0,2,...,0,0,0,0,0,0,0,0,0,0
5791,0,1,1,1,0,2,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,0,1,0,6,0,3,2,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,1,2,1,0,1,0,2,0,1,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [20]:
# Challenge: Calculate the probability of spam - the percent of spam messages in the training
# dataset. Store this value in a variable called prob_spam

In [58]:
full_train_data.CATEGORY.size

4015

In [59]:
full_train_data.CATEGORY.sum()

np.int64(1250)

In [60]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.31133250311332505


## Total Number of Words / Tokens

In [61]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4015,)

In [63]:
email_lengths[:5]

DOC_ID
0     13
1     87
2     53
3     40
4    183
dtype: object

In [64]:
total_wc = email_lengths.sum()
total_wc

np.int64(453359)

## Number of Tokens in Spam & Ham Emails

**Challenge** Create a subset of the email_lengths series that only contains the spam messages. Call the subset spam_lengths. Then count the total number of words that occur in spam emails. 

Do the same for the non-spam emails. Create a subset called ham_lengths. Then count the total number of words that occur in the ham emails. 

In [65]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1250,)

In [66]:
spam_wc = spam_lengths.sum()
spam_wc

np.int64(192656)

In [67]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [68]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [69]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

np.int64(260703)

In [70]:
spam_wc + nonspam_wc - total_wc

np.int64(0)

In [34]:
# Challenge: Can you figure out if spam emails or non-spam emails tend to be longer? Which
# category contains more tokens? Take a guess and then verify your intuition using the dataset

In [71]:
print('Average nr of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 154
Average nr of words in ham emails 94.287


## Summing the Tokens Occuring in Spam

In [72]:
full_train_features.shape

(4015, 2500)

In [73]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,0,5,4,1,26,0,5,36,2,1,...,0,1,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,1,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,1,0,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,4,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [75]:
train_spam_tokens.shape

(1250, 2500)

In [76]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [77]:
summed_spam_tokens.tail()

2495    8
2496    6
2497    3
2498    1
2499    5
dtype: object

In [42]:
# Challenge: Repeat this process for the ham messages. Sum the tokens that occur in the 
# non-spam messages. Store the values in a variable called summed_ham_tokens. 

## Summing the Tokens Occuring in Ham

In [78]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [79]:
summed_ham_tokens.shape

(2500,)

In [80]:
summed_ham_tokens.tail()

2495    26
2496    25
2497    31
2498    30
2499    22
dtype: object

In [81]:
train_ham_tokens[2499].sum() + 1

np.int64(22)

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [82]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.010551
1     0.00475
2    0.007066
3    0.010392
4    0.006718
dtype: object

In [83]:
prob_tokens_spam.sum()

np.float64(0.9999999999999961)

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [84]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

np.float64(1.0)

# P(Token) - Probability that Token Occurs 

In [85]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [86]:
prob_tokens_all.sum()

np.float64(0.9999999999999954)

# Save the Trained Model

In [87]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

**Challenge:** Create a full matrix from the sparse_test_data. Time the function call. How long does it take? Separate the features and the target values. Save these as separate .txt files: a TEST_TARGET_FILE and a TEST_FEATURE_MATRIX file. 

In [51]:
sparse_test_data.shape

(109004, 4)

In [52]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

/tmp/ipykernel_6206/1001586850.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace=True)


CPU times: user 17.3 s, sys: 108 ms, total: 17.4 s
Wall time: 17.6 s


In [53]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [54]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)